## Предобработка

Для нормального распознавания по предложениям я решила привести все сразу в относительно унифицированный вид: так как тексты новостные, там присутствуют заголовки без точек (с двумя переносами строк) – их я привела к виду предложений с точкой. То есть покрытие все-таки будет меньше, чем 93%, так как sent_tokenize нлтк и sentencizer спейси эти случаи с заголовками считал одним предложением (upd.: 92 в пересчете). Кажется все-таки логичным их считать отдельными предложениями скорее. Дальше надо избавиться от всех конечных источников с ссылками, так как они будут только захламлять и мешать. Дальше обычные '\n' были заменены пока что на один пробел, и этот пробел учитывается в предложениях, иначе чаровая индексация полетит; как сделать это более экономно я понимаю (очевидно все на символ назад смещать каждый раз), но я боюсь, что полетит индексация, а с ней итак очень много возиться приходится.

In [ ]:
!curl -L -o train  "drive.google.com/uc?export=download&id=14EmZaMcz2VY2AgzakD1u4N78EleCFINL&confirm=yes"

In [ ]:
!python -m spacy download ru_core_news_sm
import spacy

tagger = spacy.load("ru_core_news_sm")

In [ ]:
import json
import os
import re
import numpy as np
import pandas as pd
from collections import defaultdict
from zipfile import ZipFile
from pathlib import Path
from google.colab import files
# from nltk.tokenize import sent_tokenize

In [ ]:
with ZipFile("/content/train", 'r') as file:
  file.extractall(path="/content")
file.close()

In [ ]:
# ни sentencizer у spacy, ни sent_tokenize нлтк мне не нравится, потому будут мои убогие регулярки для чистки текста
bos_pattern = re.compile('(?<!\.)\n\n')
# new_sent_pattern = re.compile('.*[\.\!\?] ')
def clean_data(text):
    text = re.sub(bos_pattern, '. ', text)
    text = re.sub('\n*Источник: .*\n*$', '', text)
    text = re.sub('\n', ' ', text)
    return re.split('(?<=[^\s][.!\?])(?=\s)', text)

In [ ]:
data = []
spans = []

dir = '/content/new_data'

for pos, filename in enumerate(os.listdir(dir)):
    if filename.endswith('.json'):
        f = json.load(open(dir+'/'+filename))
        data.append(f['Text'])
        spans.append(f['Spans'])

df = pd.DataFrame(data=[data, spans], index=['Text', 'Span']).T
dropped_df = df[df['Span'].map(lambda x: bool(x))]

Проблема, которую я обнаружила, но не смогла придумать, как починить: со всякими "кв. м" деление на новые предложения; нлтк и спейси тоже не спасают от этого, но у них еще и пробелы, необходимые для чаровой разметки, убираются, поэтому я оставила свой код.

In [ ]:
overall_spans = 0
ill_span_dist_2 = 0
span_dist_1 = 0
span_dist_2 = 0
ill_texts = defaultdict(list)
ill_distances = defaultdict(list)
# idx_to_dist_2 = defaultdict(dict)
dist_to_idx_2 = defaultdict(dict)
all_dists_2 = set()

for (idx, text, spans) in dropped_df.itertuples():
    new_text = clean_data(text)
    dropped_df.loc[idx]['Text'] = new_text
    for pair in spans:
      overall_spans += 1
      fragment = text[pair[0][0]:pair[1][1]]

      distance = len(clean_data(fragment))
      all_dists_2.add(distance)
      dist_to_idx_2[distance][idx] = pair
      # idx_to_dist_2[idx] = distance

      if distance > 2:
        ill_span_dist_2 += 1
        ill_texts[idx].append(pair)
        ill_distances[idx].append(distance)

      elif distance == 1:
        span_dist_1 += 1

      elif distance == 2:
        span_dist_2 += 1

      else:
        print('?')

In [ ]:
print(f'Дистанция одно предложение: {span_dist_1}')
print(f'Дистанция два предложения: {span_dist_2}')
print(f'Всего предложений: {overall_spans}')
print((span_dist_1+span_dist_2)/overall_spans)

In [ ]:
dropped_df.to_csv('df_csv')

In [ ]:
def morph_analysis(text, corr_feats, positions):
    true_cand = text[positions[0]:positions[1]]
    doc = tagger(text)
    # print(doc, len(doc))
    candidates = []
    true_index = -1
    cand_added = False
    context = None
    for pos, word in enumerate(doc):
        if str(word) == '-':
            continue

        # print(word, str(word) != '-')
        if true_cand.find(str(word)) != -1:
            if cand_added:
                continue
            true_index = len(candidates)
            candidates.append(positions)
            cand_added = True
            # print(cand_added)
            continue

        if word.tag_ == 'NOUN' and str(word) != '-':
            # print(word)
            # print()
            if ((corr_feats['Gender'] is None or corr_feats['Gender'] == word.morph.get('Gender'))
            and (corr_feats['Number'] is None or corr_feats['Number'] == word.morph.get('Number'))):


              # print(doc[:word.i], len(str(doc[:word.i])), len(str(doc[:word.i+1])))
              begin = word.idx
              end = word.idx+len(word)
              if begin == end:
                  print('Warning: span is empty')
              candidates.append([begin, end])
        context = word
              # print("CAND:", word, f'[{begin}, {end}]')
    if not cand_added:
        true_index = len(candidates)
        candidates.append(positions)

    return candidates, true_index

В функцию подается i-ый текст, разбитый на предложения, и список спенов для i-ого текста. В функции итерация по предложениям (разбиение на два) и итерация по спенам через очередь. Фиксируется длина текста во время итерации (text_len), а также каждый раз при разбиении на два предложения подсчитывается длина конкретного кусочка (curr_len).





In [ ]:
import heapq as hq

Еще одна проблема, которую я не учла при первичной своей предобработке: нужно было сделать булевый флажок катафора; соответственно сейчас необходимо, чтобы, если вдруг вторым линейным спеном оказывается ИГ, окно было расширено (не только в левую сторону антецедент, но и в правую). То есть для катафоры я просто запускаю поиск подходящих ИГ от всего текста.

In [ ]:
def create_new_dict(sentencized_text, span_list, text_pos, idx=0):
    dictionary = {}
    hq.heapify(span_list)
    max_length = len(sentencized_text)
    text_len = 0

    for pos in range(len(sentencized_text)):
        # print(pos, text_len)
        if pos + 1 == max_length:
            break

        new_text = ''.join(str(x) for x in [sentencized_text[pos], sentencized_text[pos+1]])
        first_sent_len = len(sentencized_text[pos])
        # print(new_text)
        curr_len = len(new_text)
        check = []
        while span_list:
            cataphora = False
            f = hq.heappop(span_list) # смотрим первый спен

            if f in ill_texts[text_pos]:
                continue

            if text_len+curr_len < f[1][1] or f[0][0] < text_len:
                hq.heappush(span_list, f)
                # если индекс конца второго элемента больше
                # то брейкаемся
                break

            corr_feats = None
            new_pair = f


            if new_pair[0][0] > text_len + first_sent_len and pos+1 != max_length-1:
                # если анафорическая пара в рамках одного (второго по счету) предложении
                # то надо включить его в оба разбиения (и там, где оно второе, и там где первое)
                check.append(new_pair)

            beg_ind_1, end_ind_1 = new_pair[0][0]-text_len, new_pair[0][1]-text_len
            beg_ind_2, end_ind_2 = new_pair[1][0]-text_len, new_pair[1][1]-text_len

            word_1 = new_text[beg_ind_1:end_ind_1]
            word_2 = new_text[beg_ind_2:end_ind_2]

            morph_an = tagger(word_2)


            if len(morph_an) == 1 and (morph_an[0].tag_ in ['PRON', 'DET', 'SCONJ']):
                  # анафорические местоимения тут, кажется, должны быть все одноразмерными
                  # если нет, то должен сработать последний else
                  corr_feats = {'Gender': morph_an[0].morph.get('Gender') or None,
                                  'Number': morph_an[0].morph.get('Number') or None,
                                  'Person': morph_an[0].morph.get('Person') or None}
                  true_cand_span = [beg_ind_1, end_ind_1]
                  true_an_span = [beg_ind_2, end_ind_2]
            else:
                morph_an = tagger(word_1)
                if len(morph_an) == 1 and (morph_an[0].tag_ in ['PRON', 'DET', 'SCONJ']):
                    # с катафорой то же самое
                    cataphora = True
                    corr_feats = {'Gender': morph_an[0].morph.get('Gender') or None,
                                  'Number': morph_an[0].morph.get('Number') or None,
                                  'Person': morph_an[0].morph.get('Person') or None}
                    true_cand_span = [beg_ind_2, end_ind_2]
                    true_an_span = [beg_ind_1, end_ind_1]
                else:

                    print(f"Warning: anaphora's length > 1: text pos = {text_pos}, word = {morph_an}, span = {beg_ind_2, end_ind_2}")
                    print(f'Second word = {word_2}')
                    print(len(tagger(word_2)), tagger(word_2)[0].tag_)
                    true_cand_span = None
                    true_an_span = None

            # для анафорического местоимения записывается словарь его морфологических характеристик
            # for word in morph_an:

            #     if word.tag_ in ['PRON', 'DET']:
            #         corr_feats = {'Gender': word.morph.get('Gender') or None,
            #                       'Number': word.morph.get('Number') or None,
            #                       'Person': word.morph.get('Person') or None}
            #     else:
            #         print(beg_ind_2, end_ind_2, word)

            candidates = None
            if corr_feats is not None:
                # получения словаря всех возможных антецедентов с их спенами и индекс правильного антецедента
                if cataphora:
                    candidates, true_index = morph_analysis(new_text, corr_feats, true_cand_span)
                else:
                    candidates, true_index = morph_analysis(new_text[:end_ind_2], corr_feats, true_cand_span)
            else:
                  print(f'Warning: no candidates, text pos = {text_pos}')

            # print('Записываю в словарь')
            if candidates is not None:
                dictionary[idx] = {'TwoSents': new_text,
                                  'AnaphPos': true_an_span,
                                  'AllAnts': candidates,
                                  'IndListAnt': true_index}
            else:
                print(f'Warning: no candidates, text pos = {text_pos}')

            idx += 1

        for s in check:
              # print('pushing it back')
              # print(pos, max_length)
              hq.heappush(span_list, s)

        check = []

        text_len += len(sentencized_text[pos])

    if span_list:
        print(text_pos, 'Warning: deque is not empty', span_list)

    return dictionary

In [ ]:
def restore_data(name='df_csv', labels=['Text', 'Span']):
    new_df = pd.read_csv(name, index_col=0)
    # new_df.drop(index=['Unnamed: 0'])
    for l in labels:
        new_df[l] = new_df[l].apply(lambda x: eval(x))
    # new_df['Span'] = new_df['Span'].apply(lambda x: eval(x))
    return new_df

In [ ]:
new_df = restore_data('df_csv')

In [ ]:
new_df

In [ ]:
create_new_dict(new_df.loc[104]['Text'], new_df.loc[104]['Span'], 104)

In [ ]:
new_dict = create_new_dict(new_df.loc[13]['Text'], new_df.loc[13]['Span'], 13)

In [ ]:
new_dict

In [ ]:
test_df = pd.DataFrame(new_dict).T

In [ ]:
test_df.to_csv('test_csv')

In [ ]:
test_csv = restore_data('test_csv', labels=['AnaphPos', 'AllAnts'])

In [ ]:
new_df = restore_data()
pos = 0
final_data = []
for (idx, text, spans) in new_df.itertuples():
    if pos % 500 == 0:
        print(pos)
    dictionary = create_new_dict(text, spans, idx)
    final_data.append(dictionary)
    pos+=1

In [ ]:
# сохранение полученных предобработанных данных в json формат
with open("mydata.json", "w") as final:
    json.dump(final_data, final)

files.download('mydata.json')

## Подготовка данных

In [ ]:
!curl -L -o data  "drive.google.com/uc?export=download&id=1S2dzrZd-6Nv3B7zuETODx4ead8769DkJ&confirm=yes"

In [ ]:
import json
import torch
from torch.utils.data.dataset import Dataset
from google.colab import files

In [ ]:
f = json.load(open('/content/data'))

In [ ]:
for x in f:
    for m in x.values():
      for pos, span in enumerate(m['AllAnts']):
          if span[0] == span[1]:
              if m['TwoSents'][span[0]-1:span[1]] != '-':
                  print(m['TwoSents'][span[0]-1:span[1]])
                  print(m)

In [ ]:
print(len(f))

In [ ]:
train, dev, test = f[:2369], f[2369:2664], f[2664:]
print(len(train), len(dev), len(test))

In [ ]:
def create_final_file(f):
    all_data = []
    for x in f:
        for m in x.values():
          idx = 0
          while m['TwoSents'][idx] == ' ':
              idx += 1

          m['TwoSents'] = m['TwoSents'][idx:]
          m['AnaphPos'][0] -= idx
          m['AnaphPos'][1] -= idx
          for pos, span in enumerate(m['AllAnts']):
              if span[0] == span[1]:
                  print(m)
              check = span[::]
              check[0] -= idx
              check[1] -= idx
              while check[0]-1 >= 0 and (m['TwoSents'][check[0]-1].isalpha() or m['TwoSents'][check[0]-1] == '-'):
                print(m['TwoSents'][check[0]-1])
                check[0] -= 1

              while len(m['TwoSents']) > check[1] and (m['TwoSents'][check[1]].isalpha() or m['TwoSents'][check[1]] == '-'):
                print(m['TwoSents'][check[1]])
                check[1] += 1

              m['AllAnts'][pos] = check
          all_data.append(m)
    return all_data

train_data = create_final_file(train)
dev_data = create_final_file(dev)
test_data = create_final_file(test)
print(len(train_data), len(dev_data), len(test_data), len(train_data)+len(dev_data)+len(test_data))

In [ ]:
def data_for_dataset(all_data):
    final_data = []
    for idx, elem in enumerate(all_data):
        for pos, span in enumerate(elem['AllAnts']):
            answer = dict()
            answer['TwoSents'] = elem['TwoSents']
            answer['AnaphPos'] = elem['AnaphPos']
            answer['Ant'] = span
            answer['IsAnt'] = int(pos == elem['IndListAnt'])
            answer['OrigPos'] = idx
            final_data.append(answer)
    return final_data

In [ ]:
train_d = data_for_dataset(train_data)
dev_d = data_for_dataset(dev_data)
test_d = data_for_dataset(test_data)
print(len(train_d), len(dev_d), len(test_d), len(train_d)+len(dev_d)+len(test_d))

In [ ]:
train_d[80422]

In [ ]:
def download_data(data, filename):
    filename = filename + ".json"
    with open(filename, "w") as file:
        json.dump(data, file)
    files.download(filename)

download_data(train_d, 'train')
download_data(dev_d, 'dev')
download_data(test_d, 'test')

## Создание датасета

In [ ]:
import json
import torch
from torch.utils.data.dataset import Dataset
from google.colab import files
from zipfile import ZipFile
from pathlib import Path

In [ ]:
!curl -L -o all_data "drive.google.com/uc?export=download&id=1XxyWtqSZ4xHTqv9U8HLHFKA1b7cS2a0h&confirm=yes"

In [ ]:
with ZipFile("/content/all_data", 'r') as file:
  file.extractall(path="/content")
file.close()

In [ ]:
train = json.load(open('train.json'))
dev = json.load(open('dev.json'))
test = json.load(open('test.json'))

In [ ]:
def convert_char_to_tokens(tokenization, char_pos, sent_idx=None):
  i = 0
  offset_mapping = tokenization["offset_mapping"]
  subtoken_start, subtoken_end = None, None
  approximate = None
  while subtoken_end is None:
    if i == len(offset_mapping):
        # print(sent_idx)
        if approximate is not None:
            subtoken_end = approximate
        else:
          subtoken_end = i

    if subtoken_start is None and (char_pos[0] == offset_mapping[i][0] or char_pos[0] == offset_mapping[i][1] or char_pos[0] == (offset_mapping[i][0] + offset_mapping[i][1])//2):
      subtoken_start = i
    if subtoken_end is None and (char_pos[1] == offset_mapping[i][0] or char_pos[1] == offset_mapping[i][1] or char_pos[1] == (offset_mapping[i][0] + offset_mapping[i][1])//2):
      subtoken_end = i+1
    if approximate is None and abs(char_pos[1]-offset_mapping[i][0]) <= 2:
        approximate = i+1
    i += 1
  return [subtoken_start, subtoken_end]

In [ ]:
class ARDataset(Dataset):

    def __init__(self, data, tokenizer, label=1, device = "cuda"):
        self.data = data
        self.tokenizer = tokenizer
        self.label = label
        self.device = device

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        tokenization = self.tokenizer(item['TwoSents'], return_offsets_mapping=True)
        anaph_pos = convert_char_to_tokens(tokenization, item['AnaphPos'], sent_idx=index)
        ant_pos = convert_char_to_tokens(tokenization, item['Ant'], sent_idx=index)

        # if ant_pos[0] is None or ant_pos[1] is None:
        #     print(ant_pos, index)

        # if anaph_pos[0] is None or anaph_pos[1] is None:
        #     print(anaph_pos, index)

        answer = dict()
        answer['input_ids'] = torch.tensor(tokenization['input_ids'], dtype=torch.int64).to(self.device)
        answer['attention_mask'] = tokenization['attention_mask']
        answer['target_starts'] = torch.tensor(anaph_pos[0], dtype=torch.int64).to(self.device)
        answer['target_ends'] = torch.tensor(anaph_pos[1], dtype=torch.int64).to(self.device)
        answer['source_starts'] = torch.tensor(ant_pos[0], dtype=torch.int64).to(self.device)
        answer['source_ends'] = torch.tensor(ant_pos[1], dtype=torch.int64).to(self.device)
        answer['label'] = torch.tensor(int(item['IsAnt'] == self.label), dtype=torch.int64).to(self.device)
        answer['idx'] = torch.tensor(item['OrigPos'], dtype=torch.int64).to(self.device)

        return answer

In [ ]:
for line in train:
    if not isinstance(line['Ant'][0], int) or not isinstance(line['Ant'][1], int):
        print(line)

## Модель RoBERTa

In [ ]:
!pip install transformers==4.28.1

In [ ]:
!pip install accelerate -U

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer
from transformers import AutoTokenizer, RobertaModel, RobertaPreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
from transformers.optimization import AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
import numpy as np
import pandas as pd
import scipy

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruRoberta-large")

In [ ]:
tokenization = tokenizer(train[0]['TwoSents'], return_offsets_mapping=True)
print(tokenization)

In [ ]:
train_dataset = ARDataset(train, tokenizer)
dev_dataset = ARDataset(dev, tokenizer)
test_dataset = ARDataset(test, tokenizer)

# train_dataset = ARDataset(train_d, tokenizer)
# dev_dataset = ARDataset(dev_d, tokenizer)
# test_dataset = ARDataset(test_d, tokenizer)

In [ ]:
for line in train_dataset:
    if line['ant_start'] == None:
        print(line)
    # print(line['ant_start'] == None)
    # break

In [ ]:
for key, value in train_dataset[0].items():
    print(key, value, sep="\n")

In [ ]:
class RobertaClassificationHead(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(4*config.hidden_size, config.hidden_size)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.out_proj = nn.Linear(config.hidden_size, 2)

    def forward(self, first, second):
        vectors = torch.cat([first, second, first*second, torch.abs(first-second)], dim=-1)
        vectors = self.dense(vectors)
        vectors = torch.tanh(vectors)
        vectors = self.dropout(vectors)
        vectors = self.out_proj(vectors)
        return vectors


class RobertaForAnaphoraClassification(RobertaPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = 2
        self.config = config
        self.roberta = RobertaModel(config, add_pooling_layer=False)
        self.classifier = RobertaClassificationHead(config)
        self.post_init()

    def forward(self, input_ids, source_starts, source_ends, target_starts, target_ends, labels=None, return_dict=False, **kwargs):
        encoder_output = self.roberta(input_ids)['last_hidden_state']
        source_encoder_output = torch.stack([torch.mean(elem[start:end], dim=0) for elem, start, end in zip(encoder_output, source_starts, source_ends)])
        target_encoder_output = torch.stack([torch.mean(elem[start:end], dim=0) for elem, start, end in zip(encoder_output, target_starts, target_ends)])
        logits = self.classifier(source_encoder_output, target_encoder_output)
        loss = None
        if labels is not None:
            labels = labels.to(logits.device)
            loss_func = nn.CrossEntropyLoss()
            loss = loss_func(logits.view(-1, self.num_labels), labels.view(-1))
        if not return_dict:
            return (logits,) if loss is None else (loss, logits)

        return SequenceClassifierOutput(loss=loss, logits=logits)

In [ ]:
roberta_model = RobertaForAnaphoraClassification.from_pretrained("ai-forever/ruRoberta-large", num_labels=2).to("cuda")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    pred_labels = np.argmax(logits, axis=-1)
    are_equal = (pred_labels == labels)
    TP = np.sum(are_equal * labels)
    FP = np.sum((1.0 - are_equal) * (1-labels))
    FN = np.sum((1.0 - are_equal) * (1-pred_labels))
    correct, total = np.sum(are_equal), len(labels)
    return {"Accuracy": 100 * correct / total, "P": 100 * TP / max(TP+FP, 1.0), "R": 100 * TP / max(TP+FN, 1.0), "F1": 100 * TP / max(TP+0.5*FN+0.5*FP, 1.0)}

In [ ]:
optimizer = AdamW(roberta_model.parameters(), lr=1e-5, weight_decay=0.01)

In [ ]:
training_args = TrainingArguments(output_dir="trainer_logs",
                                  evaluation_strategy="epoch", save_strategy='epoch', num_train_epochs=1,
                                  load_best_model_at_end=True, disable_tqdm=False,
                                  per_device_train_batch_size=4, warmup_ratio=0.1,
                                  gradient_accumulation_steps=4,
                                  metric_for_best_model="Accuracy")

In [ ]:
test_train_dataset = ARDataset(train_d[:10], tokenizer)
test_dev_dataset = ARDataset(dev_d[:10], tokenizer)

In [ ]:
trainer = Trainer(
    model=roberta_model,
    optimizers=(optimizer,None),
    args=training_args,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics)
trainer.train()

In [ ]:
training_args = TrainingArguments(output_dir="trainer_logs",
                                  evaluation_strategy="steps", save_strategy='steps',
                                  logging_strategy="steps", num_train_epochs=2,
                                  load_best_model_at_end=True, disable_tqdm=False,
                                  per_device_train_batch_size=4, warmup_ratio=0.1,
                                  gradient_accumulation_steps=4,
                                  metric_for_best_model="Accuracy", eval_steps=500)

In [ ]:
trainer = Trainer(
    model=roberta_model,
    optimizers=(optimizer,None),
    args=training_args,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics)
trainer.train()

In [ ]:
test[1]

In [ ]:
from collections import defaultdict

In [ ]:
def predict_with_trainer(trainer, test):
    same_sample = None
    count = defaultdict(int)
    correct, total = 0, 0
    for pos, elem in enumerate(test):
        if pos % 2500 == 0:
            print(pos)
        if same_sample is None:
            same_sample = []
            same_sample.append(elem)
            # это, наверно, не очень красиво
            # но пользуюсь тем, что знаю, что одинаковые предложения идут все подряд
            curr_idx = elem['OrigPos']
            continue
        if elem['OrigPos'] == curr_idx:
            same_sample.append(elem)

        else:
          if len(same_sample) > 1:
              test_dataset = ARDataset(same_sample, tokenizer)
              predictions = trainer.predict(test_dataset)

              pred = np.argmax(predictions[0][:,1])

              if same_sample[pred]['IsAnt'] == 1:
                  correct += 1
              total += 1
          count[len(same_sample)] += 1
          same_sample = []
          same_sample.append(elem)
          curr_idx = elem['OrigPos']
    if len(same_sample) > 1:
        test_dataset = ARDataset(same_sample, tokenizer)
        predictions = trainer.predict(test_dataset)

        pred = np.argmax(predictions[0][:,1])

        if same_sample[pred]['IsAnt'] == 1:
            correct += 1
        total += 1
        count[len(same_sample)] += 1
        same_sample = []
        same_sample.append(elem)
        curr_idx = elem['OrigPos']
    return correct, total

In [ ]:
correct, total count = predict_with_trainer(trainer, test)
print(correct, total)
print(count)

In [ ]:
print(correct / total)

## Проверка модели на собственных данных

In [ ]:
!curl -L -o new_test_data "drive.google.com/uc?export=download&id=1sv8bhS7B35KshRuQCfGs3NL07xX8emD3&confirm=yes"

In [ ]:
new_test_data = json.load(open('new_test_data'))
print(len(new_test_data))

In [ ]:
check_dataset = data_for_dataset(new_test_data)

In [ ]:
print(len(check_dataset))

In [ ]:
for pos, elem in enumerate(check_dataset):
  if elem['TwoSents'][elem['Ant'][0]:elem['Ant'][1]] == '':
      print(pos)
  if elem['TwoSents'][elem['AnaphPos'][0]:elem['AnaphPos'][1]] == '':
      print(pos)

In [ ]:
def my_predict(trainer, test):
    data = []
    same_sample = None
    count = defaultdict(int)
    correct, total = 0, 0
    for pos, elem in enumerate(test):
        if same_sample is None:
            same_sample = []
            same_sample.append(elem)
            curr_idx = elem['OrigPos']
            continue

        if elem['OrigPos'] == curr_idx:
            same_sample.append(elem)

        else:
          if len(same_sample) > 1:
              test_dataset = ARDataset(same_sample, tokenizer)
              predictions = trainer.predict(test_dataset)

              pred = np.argmax(predictions[0][:,1])

              if same_sample[pred]['IsAnt'] == 1:
                  print(same_sample[0]['TwoSents'], same_sample[pred]['Ant'], predictions[0])
                  correct += 1
              total += 1
          pred_span = same_sample[pred]['Ant']
          predicted_word = same_sample[0]['TwoSents'][pred_span[0]:pred_span[1]]
          data.append((same_sample[0]['TwoSents'], predicted_word, same_sample[0]['OrigPos']))
          count[len(same_sample)] += 1
          same_sample = []
          same_sample.append(elem)
          curr_idx = elem['OrigPos']
    if len(same_sample) > 1:
          test_dataset = ARDataset(same_sample, tokenizer)
          predictions = trainer.predict(test_dataset)

          pred = np.argmax(predictions[0][:,1])

          if same_sample[pred]['IsAnt'] == 1:
                  correct += 1
          total += 1
          pred_span = same_sample[pred]['Ant']
          predicted_word = same_sample[0]['TwoSents'][pred_span[0]:pred_span[1]]
          data.append((same_sample[0]['TwoSents'], predicted_word, same_sample[0]['OrigPos']))
          count[len(same_sample)] += 1
          same_sample = []
          same_sample.append(elem)
          curr_idx = elem['OrigPos']
    return correct, total, data

In [ ]:
correct, total, data = my_predict(trainer, check_dataset)

In [ ]:
correct/total

In [ ]:
# легкие случаи
fast_check = [{'TwoSents': 'Когда Маша играла с куклой, к ней подошел ее брат.',
               'AnaphPos': [30, 33],
               'Ant': [6, 10],
               'IsAnt': 1,
               'OrigPos': 0
               },
                {'TwoSents': 'Когда Маша играла с куклой, к ней подошел ее брат.',
               'AnaphPos': [30, 33],
               'Ant': [20, 26],
               'IsAnt': 0,
                'OrigPos': 0
               },
              {'TwoSents': 'Ребята не хотели делать домашние задания по школьным предметам, поэтому они пошли гулять.',
               'AnaphPos': [72, 75],
               'Ant': [0, 6],
               'IsAnt': 1,
               'OrigPos': 1
               },
              {'TwoSents': 'Ребята не хотели делать домашние задания по школьным предметам, поэтому они пошли гулять.',
               'AnaphPos': [72, 75],
               'Ant': [33, 40],
               'IsAnt': 0,
               'OrigPos': 1
               },
              {'TwoSents': 'Ребята не хотели делать домашние задания по школьным предметам, поэтому они пошли гулять.',
               'AnaphPos': [72, 75],
               'Ant': [53, 62],
               'IsAnt': 0,
               'OrigPos': 1
               }
              ]

In [ ]:
correct, total, data = my_predict(trainer, fast_check)
correct/total

In [ ]:
my_df = pd.DataFrame(data, columns=['Text', 'Prediction', 'Original data'])

In [ ]:
real_ants = []
for line in my_df.itertuples():
    true_idx = new_test_data[line[3]]['IndListAnt']
    real = new_test_data[line[3]]['AllAnts'][true_idx]
    real_ants.append(line[1][real[0]:real[1]])

In [ ]:
my_df['Real Ant'] = real_ants
my_df = my_df.drop(columns='Original data')
my_df['Correct'] = my_df['Prediction'] == my_df['Real Ant']
my_df

## Берт

In [ ]:
!pip install transformers==4.28.1

In [ ]:
!pip install accelerate -U

In [ ]:
import torch
import numpy as np
import torch.nn as nn
from transformers import AutoTokenizer
from transformers import AutoTokenizer, BertModel, BertPreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
from transformers.optimization import AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
class BertaClassificationHead(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(4*config.hidden_size, config.hidden_size)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.out_proj = nn.Linear(config.hidden_size, 2)

    def forward(self, first, second):
        vectors = torch.cat([first, second, first*second, torch.abs(first-second)], dim=-1)
        vectors = self.dense(vectors)
        vectors = torch.tanh(vectors)
        vectors = self.dropout(vectors)
        vectors = self.out_proj(vectors)
        return vectors

In [ ]:
class BertForAnaphoraClassification(BertPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = 2
        self.config = config
        self.bert = BertModel(config, add_pooling_layer=False)
        self.classifier = BertaClassificationHead(config)
        self.post_init()

    def forward(self, input_ids, source_starts, source_ends, target_starts, target_ends, labels=None, return_dict=False, **kwargs):
        encoder_output = self.bert(input_ids)['last_hidden_state']
        source_encoder_output = torch.stack([torch.mean(elem[start:end], dim=0) for elem, start, end in zip(encoder_output, source_starts, source_ends)])
        target_encoder_output = torch.stack([torch.mean(elem[start:end], dim=0) for elem, start, end in zip(encoder_output, target_starts, target_ends)])
        logits = self.classifier(source_encoder_output, target_encoder_output)
        loss = None
        if labels is not None:
            labels = labels.to(logits.device)
            loss_func = nn.CrossEntropyLoss()
            loss = loss_func(logits.view(-1, self.num_labels), labels.view(-1))
        if not return_dict:
            return (logits,) if loss is None else (loss, logits)

        return SequenceClassifierOutput(loss=loss, logits=logits)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruBert-base")
bert_model = BertForAnaphoraClassification.from_pretrained("ai-forever/ruBert-base", num_labels=2)

In [ ]:
training_args = TrainingArguments(output_dir="trainer_logs",
                                  evaluation_strategy="epoch", save_strategy='epoch', num_train_epochs=1,
                                  load_best_model_at_end=True, disable_tqdm=False,
                                  per_device_train_batch_size=8, warmup_ratio=0.1,
                                  gradient_accumulation_steps=8,
                                  metric_for_best_model="Accuracy")

In [ ]:
train_dataset = ARDataset(train, tokenizer)
dev_dataset = ARDataset(dev, tokenizer)
test_dataset = ARDataset(test, tokenizer)

In [ ]:
optimizer = AdamW(bert_model.parameters(), lr=1e-5, weight_decay=0.01)

In [ ]:
trainer = Trainer(
    model=bert_model,
    optimizers=(optimizer,None),
    args=training_args,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics)
trainer.train()

In [ ]:
training_args = TrainingArguments(output_dir="trainer_logs",
                                  evaluation_strategy="epoch", save_strategy='epoch', num_train_epochs=2,
                                  load_best_model_at_end=True, disable_tqdm=False,
                                  per_device_train_batch_size=8, warmup_ratio=0.1,
                                  gradient_accumulation_steps=8,
                                  metric_for_best_model="Accuracy")

In [ ]:
trainer = Trainer(
    model=bert_model,
    optimizers=(optimizer,None),
    args=training_args,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics)
trainer.train(resume_from_checkpoint=True)

In [ ]:
correct, total = predict_with_trainer(trainer, test)
print(correct, total)

In [ ]:
print(correct/total)